# 2022/01/01/SAT(HappyNewYear)

datail-review 해보자

- feature_names = 높이,가로 길이 이런 것들, data = 각 featuredml 값들, target = 0,1,2...예를 들면 붓꽃의 이름을 대용한 것, target_names = 각 target이 가리키는 이름이 무엇인지?

---

model_selection 모듈은 학습 데이터와 테스트 데이터 세트를 분리하거나 교차 검증 분할 및 평가, 그리고 Estimator의 하이퍼 파라미터를 튜닝하기 위한 다양한 함수와 클래스를 제공, 전체 데이터를 학습 데이터와 테스트 데이터 세트로 분리해주는 train_test_split()부터 살펴보자

In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
iris=load_iris() # 붓꽃 데이터 세트 로딩
dt_clf=DecisionTreeClassifier()
train_data=iris.data # 데이터 세트에서 feature만으로 구성된 데이터가 ndarray
train_label=iris.target # 데이터 세트에서 label 데이터
dt_clf.fit(train_data, train_label) # 학습 수행중
pred=dt_clf.predict(train_data) # 예측 수행중 // 그런데 학습때 사용했던 train_data를 사용했음 -> 예측도 1 나올 것
print('예측도: ',accuracy_score(train_label,pred))

예측도:  1.0


- 정확도가 100% 나왔음 $\to$ 이미 학습한 학습 데이터 세트를 기반으로 예측했기 때문. 답을 알고 있는데 같은 문제를 낸 것이나 마찬가지
- 따라서 예측을 수행하는 데이터 세트는 학습을 수행한 학습용 데이터 세트가 아닌 전용의 테스트 데이터 세트여야 함.

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
dt_clf=DecisionTreeClassifier()
iris=load_iris()
# train_test_split()의 반환값은 튜플 형태이다. 순차적으로 네가지 요소들을 반환한다
X_train,X_test,y_train,y_test=train_test_split(iris.data, iris.target,test_size=0.3,random_state=121)
dt_clf.fit(X_train,y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


---

지금까지의 방법은 모델이 학습 데이터에만 과도하게 최적화되어, 실제 예측을 다른 데이터로 수행할 경우에는 예측 성능이 과도하게 떨어지는 `과적합`이 발생할 수 있다. 즉 해당 테스트 데이터에만 과적합되는 학습 모델이 만들어져 다른 테스트용 데이터가들어올 경우에는 성능이 저하된다. $\to$ 개선하기 위해 `교차검증`을 이용해 다양한 학습과 평가를 수행해야 한다.

> 교차검증?

: 본고사 치르기 전, 여러 모의고사를 치르는 것. 즉 본고사가 테스트 데이터 세트에 대해 평가하는 것이라면 모의고사는 교차 검증에서 많은 학습과 검증 세트에서 알고리즘 학습과 평가를 수행하는 것.

: 학습 데이터 세트를 검증 데이터 세트와 학습 데이터 세트로 분할하여 수행한 뒤, 모든 학습/검증 과정이 완료된 후 최종적으로 성능을 평가하기 위해 테스트 데이터 세트를 마련함.

> K fold 교차 검증?

: K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증, 평가를 반복적으로 수행 / 개괄적 과정은 교재 104 참고

- 실습해보자

In [7]:
import numpy as np

In [8]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold # 위에서는 trian_test_split을 import했었음
iris=load_iris() # 붓꽃 데이터 세트 로딩
features=iris.data
label=iris.target
dt_clf=DecisionTreeClassifier(random_state=156)
kfold=KFold(n_splits=5) # KFold 객체 생성
cv_accuracy=[] # fold set별 정확도를 담을 리스트 객체 생성
print('붓꽃 데이터 세트 크기:',features.shape[0])

붓꽃 데이터 세트 크기: 150


---

```python
kfold=KFold(n_splits=5)
```
로 KFold객체를 생성했으니 객체의 split()을 호출해 전체 붓꽃 데이터를 5개의 fold 데이터 세트로 분리하자. 붓꽃 데이터 세트 크기가 150개니 120개는 학습용, 30개는 검증 테스트 데이터 세트이다. 

In [9]:
n_iter=0
for train_index,test_index in kfold.split(features):
    # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter+=1
    # 반복 시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기 :{2}, 검증 데이터 크기 :{3}'.format(n_iter,accuracy,train_size,test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n *Conclusion* 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 :1.0, 학습 데이터 크기 :120, 검증 데이터 크기 :30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기 :120, 검증 데이터 크기 :30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기 :120, 검증 데이터 크기 :30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기 :120, 검증 데이터 크기 :30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기 :120, 검증 데이터 크기 :30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

 *Conclusion* 평균 검증 정확도: 0.9


----

- 교차 검증시마다 검증 세트의 인덱스가 달라짐을 알 수 있다. 

- 검증세트 인덱스를 살펴보면 104p에서 설명한 그림의 설명과 유사함

---

> Stratified K 폴드

: 불균형한 분포도를가진 레이블(결정 클래스) 데이터 집합을 위한 K 폴드 방식이다. 불균형한 분포도를 가진 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 또는 적어서 분포가 한쪽으로 치우치는 것을 말함

가령 대출 사기 데이터를 예측한다고 가정해보자, 이 데이터 세트는 1억건이고 수십개의 feature와 대출 사기 여부를 뜻하는 label(정상 대출0, 대출사기 : 1)로 구성돼 있다. K폴드로 랜덤하게 학습 및 테스트 세트의 인덱스를 고르더라도 레이블 값인 0과1의 비율을 제대로 반영하지 못하게 됨. 따라서 원본 데이터와 유사한 대출 사기 레이블 값의 분포를 학습/테스트 세트에도 유지하는 게 매우 중요

- ***Stratified K 폴드는 이처럼 K폴드가 레이블 데이터 집합이 원본 데이터 집합의 레이블 분포를 학습 및 테스트 세트에 제대로 분배하지 못하는 경우의 문제를 해결해줌***

붓꽃 데이터 세트를 DataFrame으로 생성하고 레이블 값의 분포도를 먼저 확인해보자

In [10]:
import pandas as pd
iris=load_iris()
iris_df=pd.DataFrame(data=iris.data,columns=iris.feature_names)
iris_df['label']=iris.target
print(iris_df['label'].value_counts(),'\n')

0    50
1    50
2    50
Name: label, dtype: int64 



- label값은 모두 50개로 분배되어 있음

In [11]:
kfold=KFold(n_splits=3)
n_iter=0
for train_index, test_index in kfold.split(iris_df):
    n_iter+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test=iris_df['label'].iloc[test_index]
    print('## 교차 검증: {}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    print('------------------------------------------------------')

## 교차 검증: 1
학습 레이블 데이터 분포:
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
------------------------------------------------------
## 교차 검증: 2
학습 레이블 데이터 분포:
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
------------------------------------------------------
## 교차 검증: 3
학습 레이블 데이터 분포:
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64
------------------------------------------------------


- 교차 검증 시마다 3개의 폴드 세트로 만들어지는 학습 레이블과 검증 레이블이 완전히 다른 값으로 추출되었다. 예를 들어 첫번째 교차 검증에서는 학습 레이블의 1,2값이 각각 50개가 추출되었고 검증 레이블의 0값이 50개 추출되었음, 즉 학습레이블은 1,2 밖에 없으므로 0의 경우는 전혀 학습하지 못함. 반대로 검증 레이블은 0밖에 없으므로 학습 모델은 절대 0을 예측하지 못함. 이런 유형으로 교차 검증 데이터 세트를 분할하면 검증 예측 정확도는 0이 될 수밖에 없다. 

- StratifiedKFold는 이렇게 KFold로 분할된 레이블 데이터 세트가 전체 레이블 값의 분포도를 반영하지 못하는 문제를 해결함. 

---

실습해보자

In [12]:
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=3)
n_iter=0

# split 메소드에 인자로 feature데이터 세트뿐만 아니라 레이블 데이터 세트도 반드시 넣어줘야함
for train_index,test_index in skf.split(iris_df,iris_df['label']):
    n_iter+=1
    label_train=iris_df['label'].iloc[train_index]
    label_test=iris_df['label'].iloc[test_index]
    print('## 교차검증: {}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())
    print('--------------------------------------------------------')

## 교차검증: 1
학습 레이블 데이터 분포: 
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
1    17
2    16
Name: label, dtype: int64
--------------------------------------------------------
## 교차검증: 2
학습 레이블 데이터 분포: 
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
2    17
1    16
Name: label, dtype: int64
--------------------------------------------------------
## 교차검증: 3
학습 레이블 데이터 분포: 
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    17
2    17
0    16
Name: label, dtype: int64
--------------------------------------------------------


- 학습 레이블과 검증 레이블 데이터 값의 분포도가 동일하게 할당됐음을 알 수 있다. 이렇게 분할이 되어야 레이블 값 0,1,2를 모두 학습할 수 있고 이에 기반해 검증을 수행할 수 있다.

- 이제 StratifiedKFold를 이용해 붓꽃 데이터를 교차 검증해보자

In [13]:
df_clf=DecisionTreeClassifier(random_state=156)
skfold=StratifiedKFold(n_splits=3)
n_iter=3
cv_accuracy=[]

# StratifiedKFol의 split() 호출시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_ondex in skfold.split(features, label):
    # split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train,X_test=features[train_index],features[test_index]
    y_train,y_test=label[train_index], label[test_index]
    # 학습 및 예측
    df_clf.fit(X_train,y_train)
    pred=dt_clf.predict(X_test)
    # 반복시마다 정확도 측정
    n_iter+=1
    accuracy=np.around(accuracy_score(y_test,pred),4)
    train_size=X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{} 교차 검증 정확도 : {}, 학습 데이터 크기 : {}, 검증 데이터 크기 : {}'.format(n_iter,accuracy,train_size,test_size))
    print('#{} 검증 세트 인덱스: {}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    # 교차 검증별 정확도 및 평균 정확도 계산
    print('\n## 교차 검증별 정확도:', np.around(cv_accuracy,4))
    print('## 평균 검증 정확도:',np.mean(cv_accuracy))


#4 교차 검증 정확도 : 0.92, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#4 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.92]
## 평균 검증 정확도: 0.92

#5 교차 검증 정확도 : 0.92, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#5 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.92 0.92]
## 평균 검증 정확도: 0.92

#6 교차 검증 정확도 : 0.92, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#6 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.92 0.92 0.92]
## 평균 검증 정확도: 0.92


----

> ### ***`교차 검증을 보다 간편하게 - cross_val_score()`***

In [34]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.datasets import load_iris

iris_data=load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data= iris_data.data
label=iris_data.target

# 성능 지표는 정확도 (accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도: ',np.round(scores,4))
print('평균 검증 정확도: ',np.round(np.mean(scores),4))

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도:  0.9667


- cv로 지정된 횟수만큼 scoring 파라미터로 지정된 평가지표로 평가 결과값을 배열로 반환

----

> ### ***`GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 동시에`***

- 하이퍼 파라미터? 머신러닝 알고리즘을 구성하는 주요 구성 요소이며, 이 값을 조정해 알고리즘의 예측 성능을 개선할 수 있음

In [42]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩하고 학습 데이터와 테스트 데이터 분리
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data,iris_data.target, test_size=0.2, random_state=121)
dtree= DecisionTreeClassifier()

# 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth' : [1,2,3], 'min_samples_split' : [2,3]}

import pandas as pd
# param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정
# rifit=True가 default이며, 이때 가장 젛은 파라미터 설정으로 재학습시킴
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)
# 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train,y_train)
#GridSearchCV 결과를 추출해 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [43]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{:4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.975000


- 인덱스 4,5rk rank_test_score가 1인 것으로 보아 공동 1위이며 예측 성능 1등을 의미함. 
- 열 4,5,6은 cv=3 이라서 열2는 그 세개의 평균을 의미

In [45]:
# GridSearchCV의 refit으로 이미 학습된 estimator 반환
estimator = grid_dtree.best_estimator_
# GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습이 필요없음
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667


- 일반적으로 학습 데이터를 GridSearchCV를 이용해 최적 하이퍼 파라미터 튜닝을 수행한 뒤에 별도의 테스트 세트에서 이를 평가하는 것이 일반적인 머신 러닝 모델 적용 방법이다.